<a href="https://colab.research.google.com/github/Genimon/LDBootcamp/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
#!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

!wget  https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv
!wget  https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv

train_file_path = pd.read_csv("train-data.tsv", delimiter = '\t', names=['label', 'message'])
test_file_path = pd.read_csv("valid-data.tsv", delimiter = '\t', names=['label', 'message'])

In [ ]:
train_file_path.head()

In [ ]:
msg_ham = train_file_path[train_file_path.label == 'ham']
msg_spam = train_file_path[train_file_path.label == 'spam']

# Create a numpy list to visualise using wordcloud

msg_ham_text = " ".join(msg_ham.message.to_numpy().tolist())
msg_spam_text = " ".join(msg_spam.message.to_numpy().tolist())

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

msg_ham_cloud = WordCloud(width= 520, height= 260, stopwords= STOPWORDS, max_font_size=50, background_color= 'black', colormap= 'Blues').generate(msg_ham_text)
plt.figure(figsize=(16,10))
plt.imshow(msg_ham_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
msg_spam_cloud = WordCloud(width= 520, height= 260, stopwords= STOPWORDS, max_font_size=50, background_color= 'black', colormap= 'Blues').generate(msg_spam_text)
plt.figure(figsize=(16,10))
plt.imshow(msg_spam_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
#Get the shape of the ham messages and spam messages


ham_msg_df = msg_ham.sample(n= len(msg_spam), random_state= 42)
spam_msg_df = msg_spam
print(ham_msg_df.shape, spam_msg_df.shape)

In [ ]:
msg_df = ham_msg_df.append(spam_msg_df).reset_index(drop=True)

In [ ]:
#mapping the word 'ham' and 'spam' to 0 and 1 respectively

msg_df['label']= msg_df['label'].map({'ham':0, 'spam': 1})

In [ ]:
msg1= msg_df['message'].values
msg2= msg_df['message'].values

In [ ]:
msg1

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words= 50, char_level=False, oov_token= '<OOV>')
tokenizer.fit_on_texts(msg_df['message'])

In [ ]:
word_index = tokenizer.word_index
word_index

In [ ]:
tot_words = len(word_index)
print('There are %s unique tokens in training data. ' %tot_words)

In [ ]:
training_sequences = tokenizer.texts_to_sequences(msg_df['message'])
training_padded = pad_sequences(training_sequences, maxlen= 70, padding= 'post', truncating= 'post')

testing_sequences = tokenizer.texts_to_sequences(test_file_path['message'])
testing_padded = pad_sequences(training_sequences, maxlen= 70, padding= 'post', truncating= 'post')

In [ ]:
print(training_padded[0])

In [ ]:
print('Shape of Training Tensor: ', training_padded.shape)
print('Shape of Testing Tensor: ', testing_padded.shape)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 24
drop_value = 0.2
n_dense = 24

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional


model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length= 70))
model.add(GlobalAveragePooling1D())
model.add(Dense(24, activation= 'relu'))
model.add(Dropout(drop_value))
model.add(Dense(1, activation= 'sigmoid'))


model.compile(
    loss= 'binary_crossentropy', 
    optimizer= 'adam', 
    metrics= ['accuracy'])

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor= 'val_loss', patience=3)

history = model.fit(training_padded, 
                    msg_df['label'], 
                   epochs= 30, 
                   validation_data= (testing_padded, test_file_path['label']),
                   callbacks= [early_stop], 
                   verbose= 2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  new_seq = tokenizer.texts_to_sequences([pred_text])
    pad = pad_sequences(new_seq, maxlen= max_len, padding= padding_type, truncating= trunc_type)
    
    answer = model.predict(pad)
    
    prediction= ""
    if answer[0] * 100 < 5:
        prediction = 'ham'
    else:
        prediction = 'spam'

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
